# Blog Components

> Reusable components for the technical blog

In [ ]:
#| default_exp blog_components

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
from typing import List, Dict, Any

__all__ = ['create_nav', 'topic_card', 'math_block', 'code_block']

## Navigation Components

In [ ]:
#| export
def create_nav_links(nav_items, current_topic=None):
    """Create navigation links with active state handling"""
    
    def is_active_link(title, topic):
        if not topic: return False
        return (title.lower().startswith(topic.lower()) or 
                (topic.lower() == "home" and title == "Home") or
                (topic.lower() == "rbe" and title == "RBE Series"))
    
    def nav_link(title, url, active=False):
        """Create a navigation link using MonsterUI Button component"""
        variant = "default" if active else "ghost"
        return Button(
            title,
            href=url,
            variant=variant,
            size="sm",
            cls="transition-colors"
        )
    
    return [nav_link(title, url, is_active_link(title, current_topic)) 
            for title, url in nav_items]



In [ ]:
from IPython.display import HTML

nav_items = [
        ('Home', '/'),
        ('RBE Series', '/rbe/'),
        ('Future Topics', '/topics/'),
        ('About', '/about/')
    ]

n_links = create_nav_links(nav_items, "home")
HTML(str(n_links))

In [ ]:
#| export
def create_brand(title="Matthew Redrup's Blog", subtitle="Ramblings on AI & Cybersecurity"):
    """Create the brand/header section"""
    return Div(
        H3(title, cls="font-bold"),
        P(subtitle, cls="text-muted-foreground hidden sm:block text-sm"),
        cls="flex flex-col"
    )

In [ ]:
from IPython.display import HTML
brand = create_brand()
HTML(str(brand))

In [ ]:
#| export
def create_theme_toggle():
    """Create theme toggle that works with MonsterUI's system"""
    return Div(
        Button("☀️", 
               onclick="""
               const franken = JSON.parse(localStorage.getItem('__FRANKEN__') || '{}');
               franken.mode = 'light';
               localStorage.setItem('__FRANKEN__', JSON.stringify(franken));
               document.documentElement.classList.remove('dark');
               """, 
               variant="ghost", size="sm", cls="transition-colors"),
        Button("🌙", 
               onclick="""
               const franken = JSON.parse(localStorage.getItem('__FRANKEN__') || '{}');
               franken.mode = 'dark';
               localStorage.setItem('__FRANKEN__', JSON.stringify(franken));
               document.documentElement.classList.add('dark');
               """, 
               variant="ghost", size="sm", cls="transition-colors"),
        cls="flex gap-1"
    )


In [ ]:
from IPython.display import HTML
toggle = create_theme_toggle()
HTML(str(toggle))

## FastHTML Navigation Component with Custom Theme Toggle

This code demonstrates how to create a modular, reusable navigation component for FastHTML applications using MonsterUI styling. The key improvements over monolithic route definitions include:

### Component Separation Benefits
- **Maintainability**: Each function has a single responsibility
- **Reusability**: Components can be used across different routes
- **Testability**: Individual components can be tested in isolation
- **Readability**: Clear separation of concerns makes code easier to understand

### Key Components

#### 1. Navigation Links (`create_nav_links`)
- Uses MonsterUI `Button` components instead of manual CSS classes
- Handles active state logic cleanly with helper functions
- Returns a list of styled navigation buttons

#### 2. Brand Section (`create_brand`)
- Separates the site title and subtitle into its own component
- Uses semantic HTML with consistent styling
- Easily customizable through parameters

#### 3. Custom Theme Toggle (`create_theme_toggle`)
- **Key Discovery**: MonsterUI uses localStorage with the key `"__FRANKEN__"` to persist theme preferences
- Manually toggles the `dark` class on `document.documentElement`
- Uses matching MonsterUI Button styling for visual consistency
- Bypasses the complex `Uk_theme_switcher` component that had alignment issues

### Theme Toggle Implementation Details

The theme toggle works by:
1. Reading the current theme state from `localStorage.getItem('__FRANKEN__')`
2. Updating the `mode` property in the stored object
3. Saving back to localStorage
4. Immediately applying the theme by adding/removing the `dark` class

This approach integrates seamlessly with MonsterUI's theme system while providing better visual alignment with navigation buttons.

### Usage
Create navigation with active page indicator
```python
nav = create_nav(topic="home")
```

Use in your FastHTML route
```python
@rt('/') def get(): return Title("My Site"), nav, Main(...)
```

This pattern can be extended to other UI components like sidebars, footers, and forms.

In [ ]:
#| export
def create_nav(topic: str = None) -> NavBar:
    "Create main navigation with MonsterUI `NavBar` for current `topic`"
    nav_items = [
        ('Home', '/'),
        ('RBE Series', '/rbe/'),
        ('Future Topics', '/topics/'),
        ('About', '/about/')
    ]
    
    # Get all components
    nav_links = create_nav_links(nav_items, topic)
    brand = create_brand()
    theme_toggle = create_theme_toggle()
    
    return NavBar(
        *nav_links,
        theme_toggle,
        brand=brand,
        cls="border-b"
    )

In [ ]:
full_nav = create_nav('home')
HTML(str(full_nav))

In [ ]:
# Test create_nav - simplified for debugging
nav = create_nav("home")
print(f"NavBar type: {type(nav)}")
print(f"NavBar children count: {len(nav.children)}")
for i, child in enumerate(nav.children):
    print(f"Child {i}: {type(child).__name__} - {getattr(child, 'cls', 'no cls')}")

# Basic test that it's a NavBar
assert hasattr(nav, 'brand'), "Should have brand attribute"
assert hasattr(nav, 'children'), "Should have children attribute"

NavBar type: <class 'fastcore.xml.FT'>
NavBar children count: 2
Child 0: FT - None
Child 1: FT - None


In [ ]:
#| export
def topic_card(title: str, desc: str, url: str, status: str = "available") -> Div:
    "Create a card for blog `title` with `desc`ription and `url`"
    status_cls = f"topic-{status}"
    
    return Div(
        H3(title),
        P(desc),
        A("Read More →", href=url, cls="topic-link") if status == "available" else Span("Coming Soon", cls="coming-soon"),
        cls=f"topic-card {status_cls}"
    )

In [ ]:
# Test topic_card
card = topic_card("Test Topic", "A test description", "/test/", "available")
test_eq(card.children[0].children[0], "Test Topic")
test_eq(card.children[1].children[0], "A test description")
test_eq(card.children[2].href, "/test/")

# Test coming soon status
card_soon = topic_card("Future", "Coming later", "/future/", "coming soon")
test_eq(card_soon.children[2].children[0], "Coming Soon")

In [ ]:
#| export
def math_block(tex: str, block: bool = True) -> Div:
    "Render LaTeX `tex` using KatexMarkdownJS"
    delim = "$$" if block else "$"
    content = f"{delim}{tex}{delim}"
    
    return Div(
        content,
        cls="marked math-content" if block else "marked math-inline"
    )

In [ ]:
# Test math_block
math = math_block("x^2 + y^2 = z^2")
test_eq(math.children[0], "$$x^2 + y^2 = z^2$$")
# FastHTML components store class in attrs dict, not directly as cls
test_eq(math.attrs.get('class'), "marked math-content")

# Test inline math
inline = math_block("E = mc^2", block=False)
test_eq(inline.children[0], "$E = mc^2$")
test_eq(inline.attrs.get('class'), "marked math-inline")

## Code Display Components

In [ ]:
#| export
def code_block(code: str, language: str = "python", title: str = None) -> Div:
    """Create a syntax-highlighted code block using FastHTML's HighlightJS"""
    code_element = Pre(
        Code(code, cls=f"language-{language}")
    )
    
    if title:
        return Div(
            Div(title, cls="code-title"),
            code_element,
            cls="code-block-container"
        )
    else:
        return Div(code_element, cls="code-block-container")